In [1]:
# %pip install py7zr
# %pip install io
# %pip install schedule
# %pip install openpyxl

In [2]:
import requests, pandas as pd, io, zipfile, os, py7zr, copy, ast, time
from datetime import datetime

In [3]:
## Download and extract .7z file from url
# URL to the .7z file
url = "https://unctadstat-api.unctad.org/bulkdownload/US.CommodityPrice_M/US_CommodityPrice_M"

# Define a path for saving the downloaded .7z file
file_path = os.path.join(os.getcwd(), 'downloaded_data.7z')

# Send a GET request to the URL
response = requests.get(url, stream=True)  # Stream=True to avoid loading the content into memory at once

# Check if the request was successful
if response.status_code == 200:
    # Write the content to the .7z file
    with open(file_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192): 
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

    # Try to extract the .7z file
    with py7zr.SevenZipFile(file_path, mode='r') as z:
        z.extractall(path=os.getcwd())
    print("Extraction complete")
else:
    print(f"Download failed with status code: {response.status_code}")
    

## Retrieves CSV file and saves as DF
    
# List all files in the current directory
extracted_files = os.listdir(os.getcwd())

# You might want to filter for a specific file type, e.g., CSV files
csv_files = [file for file in extracted_files if file.endswith('.csv')]

# Assuming you want to work with the first CSV file
if csv_files:
    df = pd.read_csv(csv_files[0])
    print(df.head())
else:
    print("No CSV files found in the extracted contents.")


Extraction complete
    Period Period Label  Commodity  \
0  1995M01    Jan. 1995  IN0001.20   
1  1995M01    Jan. 1995  IN0001.01   
2  1995M01    Jan. 1995  IN0001.02   
3  1995M01    Jan. 1995  080300.01   
4  1995M01    Jan. 1995  020100.01   

                                     Commodity Label  Prices Prices Footnote  \
0                                         All groups     NaN             NaN   
1                                           All food     NaN             NaN   
2                                               Food     NaN             NaN   
3  Bananas, Central and South America, FOT, US im...    0.35             NaN   
4  Beef, Australia/New Zealand, frozen, CIF US po...    2.12             NaN   

  Prices Missing value  
0       Not applicable  
1       Not applicable  
2       Not applicable  
3                  NaN  
4                  NaN  


In [4]:
df.head()

,Period,Period Label,Commodity,Commodity Label,Prices,Prices Footnote,Prices Missing value
0,1995M01,Jan. 1995,IN0001.20,All groups,NaN,NaN,Not applicable
1,1995M01,Jan. 1995,IN0001.01,All food,NaN,NaN,Not applicable
2,1995M01,Jan. 1995,IN0001.02,Food,NaN,NaN,Not applicable
3,1995M01,Jan. 1995,080300.01,"Bananas, Central and South America, FOT, US im...",0.35,NaN,NaN
4,1995M01,Jan. 1995,020100.01,"Beef, Australia/New Zealand, frozen, CIF US po...",2.12,NaN,NaN


In [5]:
df.iloc[-1]

Period                                                            2023M12
Period Label                                                    Dec. 2023
Commodity                                                       271100.01
Commodity Label         Natural gas, index, Europe, United States and ...
Prices                                                              95.16
Prices Footnote                                                       NaN
Prices Missing value                                                  NaN
Name: 19015, dtype: object

In [6]:
#df[df["Prices Missing value"]=="Not applicable"]["Commodity Label"].unique()

In [7]:
# groups = df["Commodity Label"].unique()
# print(len(groups))
# [print("Group : " + i) for i in groups]

In [8]:
# df = df[["Period", "Commodity Label", "Prices"]]
# df

In [9]:
# us_groups = [group for group in groups if "US" in group or "United States" in group or "North America" in group]
# us_groups = [value for index, value in enumerate(us_groups) if index not in [0, 1, 4, 5, 6, 7, 10, 11, 13]]
# us_groups

In [10]:
# df = df[df["Commodity Label"].isin(us_groups)].reset_index(drop=True)
# df = df.pivot_table(index='Period', columns='Commodity Label', values='Prices', aggfunc='first').reset_index()
# df.columns.name = None
# df.rename(columns={"Period": "Date", 
#                     df.columns[1]: "Cattle Hide (Chicago)",
#                     df.columns[2]: "Maize (US)",
#                     df.columns[3]: "Natural Gas (US)",
#                     df.columns[4]: "Soybeans (US)",
#                     df.columns[5]: "Wheat (US)"
#                    }, inplace=True)

In [11]:
# commodity_prices = copy.deepcopy(df)
# commodity_prices.head()

In [12]:
xlsx_file_path = "C:/Users/tyson/Downloads/longexp1/2024M01_IPP_HIS_BEAExportHistory.xlsx"
df_bea = pd.read_excel(xlsx_file_path)

c:\Users\tyson\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [13]:
selected_columns = [df_bea.columns[1]] + list(df_bea.columns[4:16])
df_bea = df_bea[selected_columns]

new_header = df_bea.iloc[3]  # Select the row you want as the new header
df_bea = df_bea[5:3668]  # Take the data less the header row
df_bea.columns = new_header  # Set the new header
df_bea.rename(columns={df_bea.columns[0]:"Year/Commodity"}, inplace=True) 
df_bea.columns.name = None
df_bea.reset_index(drop=True, inplace=True)  # Reset the index

df_bea.head(50)

,Year/Commodity,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.
0,All commodities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1983,–,–,–,–,–,–,–,–,85.7,–,–,85.4
2,1984,–,–,85.9,–,–,87.1,–,–,85.1,–,–,84.2
3,1985,–,–,84,–,–,84.4,–,–,83.7,–,–,83.8
4,1986,–,–,83.6,–,–,83.3,–,–,82.4,–,–,83.3
5,1987,–,–,84.3,–,–,86,–,–,86.6,–,–,88.3
6,1988,–,–,89.8,–,–,92.2,–,–,94.1,–,–,93.8
7,1989,94.9,94.6,95.2,95,95.4,95.2,94.7,94,94.4,94.6,94.2,94.4
8,1990,95.2,94.6,94.8,95.1,95.3,95.1,95.3,95.4,95.8,96.6,96.7,96.3
9,1991,96.8,96.7,96.4,96.6,96.4,96.1,95.7,96,95.6,96.6,96.7,95.7


In [14]:
def safe_literal_eval(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        # Handle the error or return the original value
        return value

df_bea["Year/Commodity"] = df_bea["Year/Commodity"].apply(safe_literal_eval)

groups = df_bea[df_bea['Year/Commodity'].apply(lambda x: isinstance(x, str))]["Year/Commodity"].unique()
years = df_bea[df_bea['Year/Commodity'].apply(lambda x: isinstance(x, int))]["Year/Commodity"].unique()

years.sort()

df = pd.DataFrame(columns=groups)
df['Date'] = [f"{year}M{str(month).zfill(2)}" for year in years for month in range(1, 13)]
last_col = df.iloc[:, -1]
df = df.iloc[:, :-1]
df.insert(0, last_col.name, last_col)
df = df.drop('Advanced Technology', axis=1)

df

,Date,All commodities,Agricultural commodities,All exports excluding food and fuels (Dec. 2010=100),All exports excluding fuels (June 2022=100),Nonagricultural commodities,"All commodities, excluding computers, & semiconductors (Dec. 2001=100)","Foods, feeds, & beverages","Agricultural foods, feeds & beverages, excluding distilled beverages",Wheat and rice,...,Jewelry (Dec. 2022=100),Nonmanufactured consumer goods (Dec. 2018=100),Consumer durables--nonmanufactured (Dec. 2023=100),Gem diamonds (Dec. 2023=100),Biotechnology (Dec. 2021=100),Life Science (Dec. 2019=100),Opto-Electronics (Dec. 2022=100),Information & Communications (Dec. 2019=100),Electronics (Dec. 2019=100),Aerospace (Dec. 2019=100)
0,1978M01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1978M02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1978M03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1978M04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1978M05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,2024M08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
560,2024M09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
561,2024M10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
562,2024M11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
commodity_years = {}
for i in df_bea["Year/Commodity"]:
    if isinstance(i, str):
        commodity_years[i] = []
        current = i
    elif isinstance(i, int):
        commodity_years[current].append(i)
    else:
        pass

removed_value = commodity_years.pop('Advanced Technology', None)

In [16]:
month_dict_of_lists = {}
for month in df_bea.columns[1:16]:
    list_of_lists = []
    current_list = []

    for val in df_bea[month]:
        if pd.notna(val):
            current_list.append(val)
        else:
            if current_list:  # if current_list is not empty
                list_of_lists.append(current_list)
                current_list = []

    # Don't forget to add the last list if the last value is not NaN
    if current_list:
        list_of_lists.append(current_list)
    
    month_dict_of_lists[month] = list_of_lists


In [17]:
df_bea.head()

,Year/Commodity,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.
0,All commodities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1983,–,–,–,–,–,–,–,–,85.7,–,–,85.4
2,1984,–,–,85.9,–,–,87.1,–,–,85.1,–,–,84.2
3,1985,–,–,84,–,–,84.4,–,–,83.7,–,–,83.8
4,1986,–,–,83.6,–,–,83.3,–,–,82.4,–,–,83.3


In [18]:
df_bea = df_bea[(df_bea['Year/Commodity'] == 2022) | df_bea['Year/Commodity'].apply(lambda x: isinstance(x, str))]
df_bea = df_bea[df_bea['Year/Commodity'] != "Advanced Technology"]
df_bea['Year/Commodity'] = df_bea['Year/Commodity'].astype(str)
df_bea = df_bea[~df_bea['Year/Commodity'].str.contains("\(Dec\. 2023=100\)")]
df_bea.reset_index(drop=True, inplace=True)
df_bea

for i in df_bea['Year/Commodity']:
    print(i)

All commodities
2022
Agricultural commodities
2022
All exports excluding food and fuels  (Dec. 2010=100)
2022
All exports excluding fuels   (June 2022=100)
2022
Nonagricultural commodities
2022
All commodities, excluding computers, & semiconductors  (Dec. 2001=100)
2022
Foods, feeds, & beverages
2022
Agricultural foods, feeds & beverages, excluding distilled beverages
2022
Wheat and rice
2022
Wheat
2022
Soybeans & other oil seeds
2022
Soybeans and soybean by-products, prior to the extraction of oil
2022
Feedstuff
2022
Corn
2022
Other agricultural foods
2022
Meat, poultry & other edible animal products (Dec. 2006=100)
2022
Fruit and fruit preparations, including fruit juices
2022
Vegetables and vegetable preparations and juices (Dec. 2018=100)
2022
Nuts & preparations   (Dec. 2014=100)
2022
Bakery & confectionery products   (Dec. 2016=100)
2022
Other foods and food preparations (lard, soft bev, spices), n.e.s.
2022
Nonagricultural foods (fish, distilled beverages)
2022
Fish and shellfis

In [19]:
# Extract the headers (commodities) and create a mapping for the new headers
commodities = df_bea[df_bea['Year/Commodity'] != '2022']['Year/Commodity'].dropna().tolist()
headers_mapping = {i: commodity for i, commodity in enumerate(commodities, start=1)}

# Filter out the rows where 'Year/Commodity' is not a year
df_bea = df_bea[df_bea['Year/Commodity'] == '2022'].reset_index(drop=True)

# Drop the 'Year/Commodity' column as we no longer need it
df_bea.drop('Year/Commodity', axis=1, inplace=True)

# Rename the columns using the headers_mapping created earlier
df_bea.rename(columns=headers_mapping, inplace=True)

# Transpose the DataFrame so that months become rows and commodities become columns
df_bea = df_bea.T

# Reset the index to turn the old column names into a column in the DataFrame
df_bea.reset_index(inplace=True)

# Rename the columns appropriately
df_bea.columns = ['Month'] + commodities

# The 'Month' column will have values from the index (Jan, Feb, etc.), extract and set it
df_bea['Month'] = df_bea['Month'].str.extract('(\w+)', expand=False)

# Now, df should have the structure you want
df_bea


,Month,All commodities,Agricultural commodities,All exports excluding food and fuels (Dec. 2010=100),All exports excluding fuels (June 2022=100),Nonagricultural commodities,"All commodities, excluding computers, & semiconductors (Dec. 2001=100)","Foods, feeds, & beverages","Agricultural foods, feeds & beverages, excluding distilled beverages",Wheat and rice,...,"Toys, shooting and sporting goods","Coins, gems, jewelry & collectibles (Dec. 2022=100)",Jewelry (Dec. 2022=100),Nonmanufactured consumer goods (Dec. 2018=100),Biotechnology (Dec. 2021=100),Life Science (Dec. 2019=100),Opto-Electronics (Dec. 2022=100),Information & Communications (Dec. 2019=100),Electronics (Dec. 2019=100),Aerospace (Dec. 2019=100)
0,Jan,148.2,238.1,112.8,–,141.4,167,240.7,241.2,298.2,...,121.5,–,–,111.2,99.3,100.5,–,102,102.7,105.4
1,Feb,153,245.1,113.7,–,146.1,172.8,247.6,248,295.1,...,121.5,–,–,120.1,97.9,101.4,–,102.3,103.4,105.6
2,Mar,159,256.1,115.1,–,151.6,179.8,259.7,260.6,357.9,...,121.5,–,–,126.1,98,100.9,–,102.7,104,106
3,Apr,160.6,259.2,116.5,–,153.2,181.7,262.3,263.3,345.5,...,122,–,–,126.1,98.2,101.1,–,103.9,104.5,106.5
4,May,164.9,265.3,116.3,–,157.4,186.9,267.8,268.9,359.1,...,122.1,–,–,126.3,98,101.2,–,104,104.4,106.7
5,Jun,166.7,265.3,115.9,100,159.3,189,266.7,268.9,362.3,...,122.9,–,–,126.3,97.4,101.5,–,103.8,104.5,106.9
6,Jul,160.5,256.8,115.7,99.8,153.2,181.6,258.5,260.4,300.3,...,130,–,–,126.1,93.5,101.8,–,103.6,105.1,107.7
7,Aug,158.3,255.4,115,99.5,151,179,257.1,258.9,296.4,...,129.8,–,–,129.5,92.7,101.2,–,102.8,105.6,107.8
8,Sep,156.8,253.3,114.4,99,149.6,177.2,255.3,256.9,299.1,...,131.9,–,–,128.5,93.1,101,–,103.1,106,108
9,Oct,156,251.9,113.3,98.4,148.8,176.4,254.5,256.1,322.3,...,137.1,–,–,128.4,92.7,100.3,–,102,105.2,108.2


In [20]:
final_df = copy.deepcopy(df_bea)
#indices_to_remove = [1, 2, 3, 4, 5, 6, 7, 8]

#columns_to_remove = df_bea.columns[indices_to_remove]
# final_df = df_bea.drop(columns_to_remove, axis=1)

for i, col in enumerate(df_bea.columns):
    print(f"{col}: {i}")

final_df

Month: 0
All commodities: 1
Agricultural commodities: 2
All exports excluding food and fuels  (Dec. 2010=100): 3
All exports excluding fuels   (June 2022=100): 4
Nonagricultural commodities: 5
All commodities, excluding computers, & semiconductors  (Dec. 2001=100): 6
Foods, feeds, & beverages: 7
Agricultural foods, feeds & beverages, excluding distilled beverages: 8
Wheat and rice: 9
Wheat: 10
Soybeans & other oil seeds: 11
Soybeans and soybean by-products, prior to the extraction of oil: 12
Feedstuff: 13
Corn: 14
Other agricultural foods: 15
Meat, poultry & other edible animal products (Dec. 2006=100): 16
Fruit and fruit preparations, including fruit juices: 17
Vegetables and vegetable preparations and juices (Dec. 2018=100): 18
Nuts & preparations   (Dec. 2014=100): 19
Bakery & confectionery products   (Dec. 2016=100): 20
Other foods and food preparations (lard, soft bev, spices), n.e.s.: 21
Nonagricultural foods (fish, distilled beverages): 22
Fish and shellfish   (Dec. 2016=100): 2

,Month,All commodities,Agricultural commodities,All exports excluding food and fuels (Dec. 2010=100),All exports excluding fuels (June 2022=100),Nonagricultural commodities,"All commodities, excluding computers, & semiconductors (Dec. 2001=100)","Foods, feeds, & beverages","Agricultural foods, feeds & beverages, excluding distilled beverages",Wheat and rice,...,"Toys, shooting and sporting goods","Coins, gems, jewelry & collectibles (Dec. 2022=100)",Jewelry (Dec. 2022=100),Nonmanufactured consumer goods (Dec. 2018=100),Biotechnology (Dec. 2021=100),Life Science (Dec. 2019=100),Opto-Electronics (Dec. 2022=100),Information & Communications (Dec. 2019=100),Electronics (Dec. 2019=100),Aerospace (Dec. 2019=100)
0,Jan,148.2,238.1,112.8,–,141.4,167,240.7,241.2,298.2,...,121.5,–,–,111.2,99.3,100.5,–,102,102.7,105.4
1,Feb,153,245.1,113.7,–,146.1,172.8,247.6,248,295.1,...,121.5,–,–,120.1,97.9,101.4,–,102.3,103.4,105.6
2,Mar,159,256.1,115.1,–,151.6,179.8,259.7,260.6,357.9,...,121.5,–,–,126.1,98,100.9,–,102.7,104,106
3,Apr,160.6,259.2,116.5,–,153.2,181.7,262.3,263.3,345.5,...,122,–,–,126.1,98.2,101.1,–,103.9,104.5,106.5
4,May,164.9,265.3,116.3,–,157.4,186.9,267.8,268.9,359.1,...,122.1,–,–,126.3,98,101.2,–,104,104.4,106.7
5,Jun,166.7,265.3,115.9,100,159.3,189,266.7,268.9,362.3,...,122.9,–,–,126.3,97.4,101.5,–,103.8,104.5,106.9
6,Jul,160.5,256.8,115.7,99.8,153.2,181.6,258.5,260.4,300.3,...,130,–,–,126.1,93.5,101.8,–,103.6,105.1,107.7
7,Aug,158.3,255.4,115,99.5,151,179,257.1,258.9,296.4,...,129.8,–,–,129.5,92.7,101.2,–,102.8,105.6,107.8
8,Sep,156.8,253.3,114.4,99,149.6,177.2,255.3,256.9,299.1,...,131.9,–,–,128.5,93.1,101,–,103.1,106,108
9,Oct,156,251.9,113.3,98.4,148.8,176.4,254.5,256.1,322.3,...,137.1,–,–,128.4,92.7,100.3,–,102,105.2,108.2


In [21]:
ag_df = final_df[["Month", "Wheat", "Corn", "Soybeans and soybean by-products, prior to the extraction of oil", "Meat, poultry & other edible animal products (Dec. 2006=100)", "Nuts & preparations   (Dec. 2014=100)", "Vegetables and vegetable preparations and juices (Dec. 2018=100)", "Fruit and fruit preparations, including fruit juices", "Fish and shellfish   (Dec. 2016=100)"]]

ag_df.rename(columns={ag_df.columns[3]: "Soybeans", 
                      ag_df.columns[4]: "Animal_Products",
                      ag_df.columns[5]: "Nuts",
                      ag_df.columns[6]: "Vegetables",
                      ag_df.columns[7]: "Fruit",
                      ag_df.columns[8]: "Fish"
                      }, inplace=True)
ag_df

C:\Users\tyson\AppData\Local\Temp\ipykernel_16912\1135777656.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ag_df.rename(columns={ag_df.columns[3]: "Soybeans",


,Month,Wheat,Corn,Soybeans,Animal_Products,Nuts,Vegetables,Fruit,Fish
0,Jan,322.1,287.9,277.6,216.1,63.3,82.1,150.4,132.7
1,Feb,316.3,300.4,305.9,218,58.2,82.1,154.8,139.5
2,Mar,394.5,340.2,329.2,217.3,58.3,83.2,149.4,140.6
3,Apr,375.3,353.7,322.6,221.2,60.3,84.6,147.6,141.2
4,May,390.4,370.5,330.7,221.8,59.4,85.4,150.9,143.5
5,Jun,392.5,349.7,339.6,220.4,58.4,87.4,154.1,126.2
6,Jul,311.3,345.3,315.6,219.9,57.9,92.5,147.2,124.4
7,Aug,301.1,327.4,319,217.1,57.2,96.1,140.5,124.3
8,Sep,307,327.1,309.2,215.1,55.8,96.4,132,126.5
9,Oct,337.4,331.8,288.9,215,54,100.7,139.4,126.4


In [25]:
# # Convert the 'Month' column to a datetime object representing the first of each month in 2024
# ag_df['Month'] = pd.to_datetime(ag_df['Month'].apply(lambda x: f"{x} 2024"))

# # Pivot the DataFrame from wide to long format
# df_long = ag_df.melt(id_vars=['Month'], var_name='Commodity', value_name='Price')

# # Optionally, save the pivoted DataFrame to a CSV file
# df_long.to_csv("C:/Users/tyson/OneDrive/1 - Documents/GitHub/maddata/data/agricultural_prices.csv", index=False)

# # Replace '/path/to/your/data_folder/pivoted_data.csv' with the actual path where you want to save the CSV.

# # Print the first few rows to confirm
# print(df_long.head())


ParserError: Unknown string format: 2024-01-01 00:00:00 2024 present at position 0

In [ ]:
ag_df

,Month,Wheat,Corn,Soybeans,Animal_Products,Nuts,Vegetables,Fruit,Fish
0,Jan,322.1,287.9,277.6,216.1,63.3,82.1,150.4,132.7
1,Feb,316.3,300.4,305.9,218,58.2,82.1,154.8,139.5
2,Mar,394.5,340.2,329.2,217.3,58.3,83.2,149.4,140.6
3,Apr,375.3,353.7,322.6,221.2,60.3,84.6,147.6,141.2
4,May,390.4,370.5,330.7,221.8,59.4,85.4,150.9,143.5
5,Jun,392.5,349.7,339.6,220.4,58.4,87.4,154.1,126.2
6,Jul,311.3,345.3,315.6,219.9,57.9,92.5,147.2,124.4
7,Aug,301.1,327.4,319,217.1,57.2,96.1,140.5,124.3
8,Sep,307,327.1,309.2,215.1,55.8,96.4,132,126.5
9,Oct,337.4,331.8,288.9,215,54,100.7,139.4,126.4


In [ ]:
# csv_file_path = os.path.join('data', 'agricultural_prices.csv')
# ag_df.to_csv(csv_file_path, index=False)

NameError: name 'convert_month_to_date' is not defined